In [1]:
import pandas as pd
import numpy as np

In [2]:
cf_df = pd.read_csv('confirmed_0330.csv')
d_df = pd.read_csv('deaths_0330.csv')
rec_df = pd.read_csv('recovered_0330.csv')

In [3]:
dates = cf_df.columns[4:]

cf_df1 = cf_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars= dates, 
    var_name='Date', 
    value_name='Confirmed'
)

d_df1 = d_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars= dates, 
    var_name='Date', 
    value_name='Deaths'
)

rec_df1 = rec_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars= dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [4]:
rec_df1 = rec_df1[rec_df1['Country/Region']!='Canada']

In [5]:


data = cf_df1.merge(
  right=d_df1, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

data = data.merge(
  right=rec_df1, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,01/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,01/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,01/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,01/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,01/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
118911,NaN,Vietnam,14.058324,108.277199,03/30/21,2594,35,2359.0
118912,NaN,West Bank and Gaza,31.952200,35.233200,03/30/21,240065,2614,213590.0
118913,NaN,Yemen,15.552727,48.516388,03/30/21,4247,882,1667.0
118914,NaN,Zambia,-13.133897,27.849332,03/30/21,88199,1202,84519.0


In [6]:
data['Date'] = pd.to_datetime(data['Date'])

In [7]:
data.isna().sum()

Province/State    82026
Country/Region        0
Lat                 434
Long                434
Date                  0
Confirmed             0
Deaths                0
Recovered          9114
dtype: int64

In [8]:
data['Recovered'] = data['Recovered'].fillna(0)

In [9]:
ship = data['Province/State'].str.contains('Grand Princess') | data['Province/State'].str.contains('Diamond Princess') | data['Country/Region'].str.contains('Diamond Princess') | data['Country/Region'].str.contains('MS Zaandam')

In [10]:
data = data[~(ship)]

In [11]:
data['Active'] = data['Confirmed'] - data['Deaths'] - data['Recovered']

<ipython-input-11-e26692325ac0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Active'] = data['Confirmed'] - data['Deaths'] - data['Recovered']


In [12]:
full = data.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
full

<ipython-input-12-68f6d96de7cc>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full = data.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
82455,2021-03-30,Vietnam,2594,35,2359.0,200.0
82456,2021-03-30,West Bank and Gaza,240065,2614,213590.0,23861.0
82457,2021-03-30,Yemen,4247,882,1667.0,1698.0
82458,2021-03-30,Zambia,88199,1202,84519.0,2478.0


In [13]:
# new cases 
temp = full.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full = pd.merge(full, temp, on=['Country/Region', 'Date'])
# filling na with 0
full = full.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full[cols] = full[cols].astype('int')
# 
full['New cases'] = full['New cases'].apply(lambda x: 0 if x<0 else x)

<ipython-input-13-9721661820de>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


In [14]:
full.to_csv('coviddata0330.csv')